<a href="https://colab.research.google.com/github/talhaanwarch/OffenseEval2020/blob/master/Greek/biGRU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Greek Language

In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [2]:
cd /content/drive/My Drive/dataset/OffenseEval2020/data/Greek/

/content/drive/My Drive/dataset/OffenseEval2020/data/Greek


In [3]:
ls

offenseval-greek-training-v1.tsv  readme-trainingset-greek.txt


In [0]:
import pandas as pd
train=pd.read_csv( 'offenseval-greek-training-v1.tsv',sep="\t")

In [5]:
train.head()

,id,tweet,subtask_a
0,1172,@USER Οι μουσουλμάνες που τις βιάζουν έτσι κ α...,OFF
1,4078,Η Κάτια προσπαθεί να πείσει οτι δεν είναι ελέφ...,NOT
2,135,"Καλά γιατί λες ότι, είσαι νέος αφού γεννήθηκες...",NOT
3,9056,Με Φατσεα ξεκινησαμε...... #Kokkinopotami,NOT
4,5344,"#gntmgr Κάτια πόσο γλυκιά, εμετός",NOT


In [6]:
train.tail()

,id,tweet,subtask_a
8738,968,Σοβαρό δημοκρατικό έλλειμμα η απουσία γυναικών...,NOT
8739,5463,Γιουροβίζιον βλέπουμε #XFactorGR,NOT
8740,7288,Στην Κάτια να δοκιμάσουμε κάτι τέτοιο παρακαλώ...,NOT
8741,5175,Γιατι δεν ειναι για μοντέλο! Γι’αυτο κερα Βίκυ...,NOT
8742,134,Κανονικά ο τύπος θα έπρεπε να ντρέπεται μόνο κ...,NOT


In [0]:
labels=train['subtask_a']
train=train['tweet']
labels=pd.factorize(labels)[0]

In [8]:
import collections
collections.Counter(labels)

Counter({0: 2486, 1: 6257})

In [9]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
max_words = 10000 #frequency of words to be kept
max_len = 200

tokenize = Tokenizer(num_words=max_words)
tokenize.fit_on_texts(train)
sequences = tokenize.texts_to_sequences(train)
word_index = tokenize.word_index
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

Using TensorFlow backend.


In [10]:
len(sequences_matrix),len(labels)

(8743, 8743)

In [0]:
from keras.models import Sequential
from keras.layers import Embedding,CuDNNGRU,GRU,Dense,Dropout,Bidirectional,GlobalMaxPool1D,GlobalAveragePooling1D, SpatialDropout1D
from keras.optimizers import RMSprop
from keras.initializers import Constant
from sklearn.utils import class_weight
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

In [0]:
from keras import backend as K
def f1(y_true, y_pred):
    '''
    metric from here 
    https://stackoverflow.com/questions/43547402/how-to-calculate-f1-macro-in-keras
    '''
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [0]:
model = Sequential()
model.add(Embedding(input_dim=max_words,output_dim=50,input_length=max_len))
model.add(Bidirectional(CuDNNGRU(64, return_sequences = True)))
#model.add(Bidirectional(CuDNNGRU(30, return_sequences = True)))
model.add(GlobalMaxPool1D())
model.add(Dense(50, activation="relu"))
model.add(Dense(20, activation="relu"))

model.add(Dropout(0.05))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss='binary_crossentropy', optimizer='Rmsprop', metrics=[f1])

In [0]:
import numpy as np
class_weights = class_weight.compute_class_weight('balanced',np.unique(labels),labels)
class_weights=dict(enumerate(class_weights))

In [15]:

model.fit(sequences_matrix,labels,batch_size=256,epochs=10,verbose=2,
          validation_split=0.2,class_weight=class_weights)

Train on 6994 samples, validate on 1749 samples
Epoch 1/10
 - 3s - loss: 0.6862 - f1: 0.5009 - val_loss: 0.6756 - val_f1: 0.8455
Epoch 2/10
 - 1s - loss: 0.6623 - f1: 0.7565 - val_loss: 0.5474 - val_f1: 0.8165
Epoch 3/10
 - 1s - loss: 0.4460 - f1: 0.8627 - val_loss: 0.5001 - val_f1: 0.8384
Epoch 4/10
 - 1s - loss: 0.3256 - f1: 0.9161 - val_loss: 0.6071 - val_f1: 0.8872
Epoch 5/10
 - 1s - loss: 0.2435 - f1: 0.9402 - val_loss: 0.7682 - val_f1: 0.8802
Epoch 6/10
 - 1s - loss: 0.1954 - f1: 0.9515 - val_loss: 0.7786 - val_f1: 0.8790
Epoch 7/10
 - 1s - loss: 0.1772 - f1: 0.9489 - val_loss: 0.7628 - val_f1: 0.8502
Epoch 8/10
 - 1s - loss: 0.1117 - f1: 0.9748 - val_loss: 0.8423 - val_f1: 0.8461
Epoch 9/10
 - 1s - loss: 0.0906 - f1: 0.9795 - val_loss: 0.9449 - val_f1: 0.8201
Epoch 10/10
 - 1s - loss: 0.0731 - f1: 0.9838 - val_loss: 1.0761 - val_f1: 0.8450


In [0]:
from sklearn.model_selection import train_test_split
x_train,x_val,y_train,y_val=train_test_split(sequences_matrix,labels,test_size=0.2,stratify=labels,shuffle=True)

In [17]:
from sklearn.metrics import classification_report

y_pred = model.predict(x_val, batch_size=128, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)
y_pred = (y_pred > 0.5)

print(classification_report(y_val, y_pred))

1749/1749 [==============================] - 0s 97us/step
              precision    recall  f1-score   support

           0       0.90      0.92      0.91       497
           1       0.97      0.96      0.96      1252

    accuracy                           0.95      1749
   macro avg       0.93      0.94      0.94      1749
weighted avg       0.95      0.95      0.95      1749



In [0]:
import keras
keras.backend.clear_session()